In [1]:
!nvidia-smi


Fri Jan 12 14:57:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm
import timm
from torch import nn
from collections import Counter
import plotly.express as px
from torch.optim import Adam

c:\Users\MSI\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from google.colab import files
import zipfile

# Téléchargez le fichier zip depuis votre ordinateur local
uploaded = files.upload()

# Spécifiez le nom du fichier zip
Database_zip = list(uploaded.keys())[0]

# Extrayez le contenu du fichier zip dans le répertoire de destination
with zipfile.ZipFile(Database_zip, 'r') as zip_ref:
    zip_ref.extractall("Database")

Saving archive.zip to archive.zip


In [2]:
train_dataset_path ='C:/Users/MSI/OneDrive/Desktop/3 ING/Semestre 1/Indexation/Mini Projet/Travail Mini Projet/archive/train' 
valid_dataset_path = 'C:/Users/MSI/OneDrive/Desktop/3 ING/Semestre 1/Indexation/Mini Projet/Travail Mini Projet/archive/test'

In [3]:
LR = 0.001
EPOCHS = 20
BATCH_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Data Augmentations
train_augs = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=(-20, +20)),
    T.ToTensor()
])

valid_augs = T.Compose([T.ToTensor()])

In [5]:
# Load Datasets
trainset = ImageFolder(train_dataset_path, transform=train_augs)
validset = ImageFolder(valid_dataset_path, transform=valid_augs)

In [6]:
import plotly.express as px

def plot_dist(my_dict, data):
    fig = px.bar(x=my_dict.values(),
             y=my_dict.keys(),
             orientation='h',
             color=my_dict.values(),
             title=data + ' Distribution',
             opacity=0.8,
             color_discrete_sequence=px.colors.diverging.curl,
             template='plotly_dark'
            )
    fig.update_xaxes()
    fig.show()

In [7]:
# Plot Class Distribution
labels = [label for _, label in trainset]
class_counts = Counter(labels)
plot_dist(class_counts, 'Trainset')

In [30]:
from torch.utils.data import DataLoader
# Data Loaders
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
validloader = DataLoader(validset, batch_size=BATCH_SIZE)

In [31]:
# VGG Model
class VGGModel(nn.Module):
    def __init__(self):
        super(VGGModel, self).__init__()
        self.vgg_net = timm.create_model('vgg16', pretrained=True, num_classes=7)

    def forward(self, images, labels=None):
        logits = self.vgg_net(images)
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            return logits, loss
        return logits


In [32]:
# Model, Optimizer, and Loss Function
vgg_model = VGGModel()
vgg_model.to(DEVICE)
vgg_optimizer = Adam(vgg_model.parameters(), lr=LR)

In [33]:
# Training and Evaluation Functions
def train_fn(model, dataloader, optimizer, current_epoch):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    tk = tqdm(dataloader, desc="EPOCHS" + "[TRAIN]" + str(current_epoch + 1) + "/" + str(EPOCHS))
    for t, data in enumerate(tk):
        images, labels = data
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        logits, loss = model(images, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits, labels)
        tk.set_postfix({'loss': '%6f' % float(total_loss / (t + 1)), 'acc': '%6f' % float(total_acc / (t + 1))})
    return total_loss / len(dataloader), total_acc / len(dataloader)

In [34]:
def eval_fn(model, dataloader, current_epoch):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    tk = tqdm(dataloader, desc="EPOCH" + "[VALID]" + str(current_epoch + 1) + "/" + str(EPOCHS))
    for t, data in enumerate(tk):
        images, labels = data
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        logits, loss = model(images, labels)

        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits, labels)
        tk.set_postfix({'loss': '%6f' % float(total_loss / (t + 1)), 'acc': '%6f' % float(total_acc / (t + 1))})
    return total_loss / len(dataloader), total_acc / len(dataloader)


In [23]:
def multiclass_accuracy(y_pred,y_true):
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [36]:
# Training Loop
best_valid_losses = {'vgg': np.inf}

for epoch in range(EPOCHS):
    vgg_train_loss, vgg_train_acc = train_fn(vgg_model, trainloader, vgg_optimizer, epoch)
    vgg_eval_loss, vgg_eval_acc = eval_fn(vgg_model, validloader, epoch)

    # Print progress
    print(f'Epoch [{epoch + 1}/{EPOCHS}], '
          f'VGG Train Loss: {vgg_train_loss:.4f}, Train Acc: {vgg_train_acc:.4f}, '
          f'VGG Valid Loss: {vgg_eval_loss:.4f}, Valid Acc: {vgg_eval_acc:.4f}')

    # Check for improvement in validation losses
    if vgg_eval_loss < best_valid_losses['vgg']:
        torch.save(vgg_model.state_dict(), '/content/modelVgg.pt')
        print("Saved Best VGG Valid Loss")
        best_valid_losses['vgg'] = vgg_eval_loss

print(best_valid_losses['vgg'])

EPOCH[VALID]1/20: 100%|██████████| 225/225 [00:15<00:00, 14.69it/s, loss=1.815587, acc=0.246389]


Epoch [1/20], VGG Train Loss: 1.8124, Train Acc: 0.2515, VGG Valid Loss: 1.8156, Valid Acc: 0.2464
Saved Best VGG Valid Loss


EPOCH[VALID]2/20: 100%|██████████| 225/225 [00:15<00:00, 14.68it/s, loss=1.814806, acc=0.246389]


Epoch [2/20], VGG Train Loss: 1.8123, Train Acc: 0.2513, VGG Valid Loss: 1.8148, Valid Acc: 0.2464
Saved Best VGG Valid Loss


EPOCH[VALID]3/20: 100%|██████████| 225/225 [00:15<00:00, 14.64it/s, loss=1.815141, acc=0.246389]


...

Epoch [18/20], VGG Train Loss: 1.8105, Train Acc: 0.2515, VGG Valid Loss: 1.8158, Valid Acc: 0.2464


EPOCH[VALID]19/20: 100%|██████████| 225/225 [00:14<00:00, 15.03it/s, loss=1.815306, acc=0.246389]


Epoch [19/20], VGG Train Loss: 1.8103, Train Acc: 0.2515, VGG Valid Loss: 1.8153, Valid Acc: 0.2464


EPOCH[VALID]20/20: 100%|██████████| 225/225 [00:14<00:00, 15.02it/s, loss=1.814501, acc=0.246389]

Epoch [20/20], VGG Train Loss: 1.8104, Train Acc: 0.2511, VGG Valid Loss: 1.8145, Valid Acc: 0.2464
1.8142721875508627
